## **Applio NoUI**
A simple, high-quality voice conversion tool focused on ease of use and performance.

[Support](https://discord.gg/urxFjYmYYh) — [GitHub](https://github.com/IAHispano/Applio) — [Terms of Use](https://github.com/IAHispano/Applio/blob/main/TERMS_OF_USE.md)

<br>

---

<br>

#### **Acknowledgments**

To all external collaborators for their special help in the following areas:
* Hina (Encryption method)
* Poopmaster (Extra section)
* Shirou (UV installer)
* Bruno5430 (AutoBackup code and general notebook maintenance)

#### **Disclaimer**
By using Applio, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while Applio disclaims liability and reserves the right to amend these terms.

### **Install Applio**
If the runtime restarts, re-run the installation steps.

In [ ]:
# @title Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

# Migrate folders to match documentation
from pathlib import Path
if Path("/content/drive").is_mount():
  %cd "/content/drive/MyDrive/"
  if not Path("ApplioBackup/").exists() and Path("RVC_Backup/").exists():
    !mv "RVC_Backup/" "ApplioBackup/"
  %cd /content

In [ ]:
# @title Setup runtime environment
from multiprocessing import cpu_count
cpu_cores = cpu_count()
post_process = False
hop_length = 128 # Common default value for hop_length, added for consistency across cells
LOGS_FOLDER = "/content/Applio/logs"
BACKUPS_PATH = "/content/drive/MyDrive/ApplioBackup"

!git config --global advice.detachedHead false
!git clone https://github.com/IAHispano/Applio --branch 3.2.9 --single-branch
%cd /content/Applio

!sudo update-alternatives --set python3 /usr/bin/python3.10
!curl -LsSf https://astral.sh/uv/install.sh | sh

print("Installing requirements...")
!uv pip install -q -r requirements.txt
!python core.py "prerequisites" --models "True" --pretraineds_hifigan "True"
print("Finished installing requirements!")!rm -r "{LOGS_FOLDER}/zips"

### **Infer**


In [ ]:
# @title Download model
# @markdown Hugging Face or Google Drive
model_link = "https://huggingface.co/Darwin/Darwin/resolve/main/Darwin.zip"  # @param {type:"string"}

%cd /content/Applio
!python core.py download --model_link "{model_link}"

In [ ]:
# @title Run Inference
# @markdown Please upload the audio file to your Google Drive path `/content/drive/MyDrive` and specify its name here. For the model name, use the zip file name without the extension. Alternatively, you can check the path `/content/Applio/logs` for the model name (name of the folder).
%cd /content/Applio
from pathlib import Path

model_name = "Darwin"  # @param {type:"string"}
model_path = Path(f"{LOGS_FOLDER}/{model_name}")
if not (model_path.exists() and model_path.is_dir()):
    raise FileNotFoundError(f"Model directory not found: {model_path.resolve()}")

# Select either the last checkpoint or the final weight
!ls -t "{model_path}"/"{model_name}"_*e_*s.pth "{model_path}"/"{model_name}".pth 2> /dev/null | head -n 1 > /tmp/pth.txt
pth_file = open("/tmp/pth.txt", "r").read().strip()
if pth_file == "":
  raise FileNotFoundError(f"No model weight found in directory: {model_path.resolve()}\nMake sure that the file is properly named (e.g. \"{model_name}.pth)\"")

!ls -t "{model_path}"/*.index | head -n 1 > /tmp/index.txt
index_file = open("/tmp/index.txt", "r").read().strip()

input_path = "/content/example.wav"  # @param {type:"string"}
output_path = "/content/output.wav"
export_format = "WAV"  # @param ['WAV', 'MP3', 'FLAC', 'OGG', 'M4A'] {allow-input: false}
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
f0_up_key = 0  # @param {type:"slider", min:-24, max:24, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
f0_autotune = False  # @param{type:"boolean"}
formant_shift = False # @param{type:"boolean"}
formant_qfrency = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
formant_timbre = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
embedder_model = "contentvec" # @param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"] {allow-input: false}
embedder_model_custom = "" # @param {type:"string"}

!rm -f "{output_path}"
if post_process:
  !python core.py infer --pitch "{f0_up_key}" --volume_envelope "{rms_mix_rate}" --index_rate "{index_rate}" --hop_length "{hop_length}" --protect "{protect}" --f0_autotune "{f0_autotune}" --f0_method "{f0_method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}" --formant_shifting "{formant_shift}" --formant_qfrency "{formant_qfrency}" --formant_timbre "{formant_timbre}" --post_process "{post_process}" --reverb "{reverb}" --pitch_shift "{pitch_shift}" --limiter "{limiter}" --gain "{gain}" --distortion "{distortion}" --chorus "{chorus}" --bitcrush "{bitcrush}" --clipping "{clipping}" --compressor "{compressor}" --delay "{delay}" --reverb_room_size "{reverb_room_size}" --reverb_damping "{reverb_damping}" --reverb_wet_gain "{reverb_wet_gain}" --reverb_dry_gain "{reverb_dry_gain}" --reverb_width "{reverb_width}" --reverb_freeze_mode "{reverb_freeze_mode}" --pitch_shift_semitones "{pitch_shift_semitones}" --limiter_threshold "{limiter_threshold}" --limiter_release_time "{limiter_release_time}" --gain_db "{gain_db}" --distortion_gain "{distortion_gain}" --chorus_rate "{chorus_rate}" --chorus_depth "{chorus_depth}" --chorus_center_delay "{chorus_center_delay}" --chorus_feedback "{chorus_feedback}" --chorus_mix "{chorus_mix}" --bitcrush_bit_depth "{bitcrush_bit_depth}" --clipping_threshold "{clipping_threshold}" --compressor_threshold "{compressor_threshold}" --compressor_ratio "{compressor_ratio}" --compressor_attack "{compressor_attack}" --compressor_release "{compressor_release}" --delay_seconds "{delay_seconds}" --delay_feedback "{delay_feedback}" --delay_mix "{delay_mix}"
else:
  !python core.py infer --pitch "{f0_up_key}" --volume_envelope "{rms_mix_rate}" --index_rate "{index_rate}" --hop_length "{hop_length}" --protect "{protect}" --f0_autotune "{f0_autotune}" --f0_method "{f0_method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}" --formant_shifting "{formant_shift}" --formant_qfrency "{formant_qfrency}" --formant_timbre "{formant_timbre}" --post_process "{post_process}"

if Path(output_path).exists():
  from IPython.display import Audio, display
  output_path = output_path.replace(".wav", f".{export_format.lower()}")
  display(Audio(output_path, autoplay=True))

In [ ]:
# @title Enable post-processing effects for inference
post_process = False # @param{type:"boolean"}
reverb = False # @param{type:"boolean"}
pitch_shift = False # @param{type:"boolean"}
limiter = False # @param{type:"boolean"}
gain = False # @param{type:"boolean"}
distortion = False # @param{type:"boolean"}
chorus = False # @param{type:"boolean"}
bitcrush = False # @param{type:"boolean"}
clipping = False # @param{type:"boolean"}
compressor = False # @param{type:"boolean"}
delay = False # @param{type:"boolean"}

reverb_room_size = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_damping = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_wet_gain = 0.0 # @param {type:"slider", min:-20.0, max:20.0, step:0.1}
reverb_dry_gain = 0.0 # @param {type:"slider", min:-20.0, max:20.0, step:0.1}
reverb_width = 1.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_freeze_mode = 0.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

pitch_shift_semitones = 0.0 # @param {type:"slider", min:-12.0, max:12.0, step:0.1}

limiter_threshold = -1.0 # @param {type:"slider", min:-20.0, max:0.0, step:0.1}
limiter_release_time = 0.05 # @param {type:"slider", min:0.0, max:1.0, step:0.01}

gain_db = 0.0 # @param {type:"slider", min:-20.0, max:20.0, step:0.1}

distortion_gain = 0.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

chorus_rate = 1.5 # @param {type:"slider", min:0.1, max:10.0, step:0.1}
chorus_depth = 0.1 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
chorus_center_delay = 15.0 # @param {type:"slider", min:0.0, max:50.0, step:0.1}
chorus_feedback = 0.25 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
chorus_mix = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

bitcrush_bit_depth = 4 # @param {type:"slider", min:1, max:16, step:1}

clipping_threshold = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

compressor_threshold = -20.0 # @param {type:"slider", min:-60.0, max:0.0, step:0.1}
compressor_ratio = 4.0 # @param {type:"slider", min:1.0, max:20.0, step:0.1}
compressor_attack = 0.001 # @param {type:"slider", min:0.0, max:0.1, step:0.001}
compressor_release = 0.1 # @param {type:"slider", min:0.0, max:1.0, step:0.01}

delay_seconds = 0.1 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
delay_feedback = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
delay_mix = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}


### **Train**

In [ ]:
# @title Setup model parameters
model_name = "Darwin" # @param {type:"string"}
sample_rate = "40k"  # @param ["32k", "40k", "48k"] {allow-input: false}
sr = int(sample_rate.rstrip("k")) * 1000

# @markdown 💾 Automatically Save/Load model from your mounted drive
auto_backups = True  # @param{type:"boolean"}

if auto_backups:
  from pathlib import Path
  if Path("/content/drive").is_mount():
    model_folder = Path(f"{LOGS_FOLDER}/{model_name}")
    if model_folder.exists():
      if model_folder.is_dir(follow_symlinks=False):
        if not Path(f"{BACKUPS_PATH}/{model_name}").exists():
          !mkdir -p "{BACKUPS_PATH}/{model_name}"
          !mv "{model_folder.resolve()}" "{BACKUPS_PATH}/{model_name}"
        else:
          raise FileExistsError(f"'{model_folder.resolve()}' conflicts with '{BACKUPS_PATH}/{model_name}'\nPlease delete one of them to enable automatic backup save/load.")
      else:
        !rm "{model_folder.resolve()}"
    !mkdir -p "{BACKUPS_PATH}/{model_name}"
    !ln -s "{BACKUPS_PATH}/{model_name}" "{LOGS_FOLDER}"
  else:
    print("Drive not mounted, skipping backup save/load")

In [ ]:
# @title Preprocess Dataset

dataset_path = "/content/drive/MyDrive/Darwin_Dataset"  # @param {type:"string"}

cut_preprocess = "Automatic" # @param ["Skip","Simple","Automatic"]
chunk_len = 3 # @param {type:"slider", min:0.5, max:5.0, step:0.5}
overlap_len = 0.3 # @param {type:"slider", min:0, max:0.5, step:0.1}
process_effects = False # @param{type:"boolean"}
noise_reduction = False # @param{type:"boolean"}
noise_reduction_strength = 0.7 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

%cd /content/Applio
!python core.py preprocess --model_name "{model_name}" --dataset_path "{dataset_path}" --sample_rate "{sr}" --cpu_cores "{cpu_cores}" --cut_preprocess "{cut_preprocess}" --process_effects "{process_effects}" --noise_reduction "{noise_reduction}" --noise_reduction_strength "{noise_reduction_strength}" --chunk_len "{chunk_len}" --overlap_len "{overlap_len}"

In [ ]:
# @title Extract Features
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe"] {allow-input: false}

sr = int(sample_rate.rstrip("k")) * 1000
include_mutes = 2 # @param {type:"slider", min:0, max:10, step:1}
embedder_model = "contentvec" # @param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"] {allow-input: false}
embedder_model_custom = "" # @param {type:"string"}

%cd /content/Applio
!python core.py extract --model_name "{model_name}" --f0_method "{f0_method}" --hop_length "{hop_length}" --sample_rate "{sr}" --cpu_cores "{cpu_cores}" --gpu "0" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}"  --include_mutes "{include_mutes}"

In [ ]:
# @title Generate index file
index_algorithm = "Auto"  # @param ["Auto", "Faiss", "KMeans"] {allow-input: false}

%cd /content/Applio
!python core.py index --model_name "{model_name}" --index_algorithm "{index_algorithm}"

In [ ]:
# @title Start Training
# @markdown ### ⚙️ Train Settings
total_epoch = 800  # @param {type:"integer"}
batch_size = 8  # @param {type:"slider", min:1, max:25, step:0}
pretrained = True  # @param{type:"boolean"}
cleanup = False  # @param{type:"boolean"}
cache_data_in_gpu = False  # @param{type:"boolean"}
vocoder = "HiFi-GAN" # @param ["HiFi-GAN"]
checkpointing = False
tensorboard = True  # @param{type:"boolean"}
# @markdown ### ➡️ Choose how many epochs your model will be stored
save_every_epoch = 10  # @param {type:"slider", min:1, max:100, step:0}
save_only_latest = False  # @param{type:"boolean"}
save_every_weights = False  # @param{type:"boolean"}
overtraining_detector = False  # @param{type:"boolean"}
overtraining_threshold = 50  # @param {type:"slider", min:1, max:100, step:0}
# @markdown ### ❓ Optional
# @markdown In case you select custom pretrained, you will have to download the pretraineds and enter the path of the pretraineds.
custom_pretrained = False  # @param{type:"boolean"}
g_pretrained_path = "/content/Applio/rvc/models/pretraineds/pretraineds_custom/G48k.pth"  # @param {type:"string"}
d_pretrained_path = "/content/Applio/rvc/models/pretraineds/pretraineds_custom/D48k.pth"  # @param {type:"string"}


%cd /content/Applio
if tensorboard:
  %load_ext tensorboard
  %tensorboard --logdir logs --bind_all
!python core.py train --model_name "{model_name}" --save_every_epoch "{save_every_epoch}" --save_only_latest "{save_only_latest}" --save_every_weights "{save_every_weights}" --total_epoch "{total_epoch}" --sample_rate "{sr}" --batch_size "{batch_size}" --gpu 0 --pretrained "{pretrained}" --custom_pretrained "{custom_pretrained}" --g_pretrained_path "{g_pretrained_path}" --d_pretrained_path "{d_pretrained_path}" --overtraining_detector "{overtraining_detector}" --overtraining_threshold "{overtraining_threshold}" --cleanup "{cleanup}" --cache_data_in_gpu "{cache_data_in_gpu}" --vocoder "{vocoder}" --checkpointing "{checkpointing}"

In [ ]:
# @title Export model
# @markdown Export model to a zip file
# @markdown * Training: Bigger file size, can continue training
# @markdown * Inference: Smaller file size, only for model inference
EXPORT_PATH = "/content/drive/MyDrive/ApplioExported"
from pathlib import Path

export_for = "training" # @param ["training", "inference"] {allow-input: false}

logs_folder = Path(f"/content/Applio/logs/{model_name}/")
if not (logs_folder.exists() and logs_folder.is_dir()):
    raise FileNotFoundError(f"{model_name} model folder not found")

%cd {logs_folder}/..
if export_for == "training":
  !zip -r "/content/{model_name}.zip" "{model_name}"
else:
  # find latest trained weight file
  !ls -t "{model_name}/{model_name}"_*e_*s.pth | head -n 1 > /tmp/weight.txt
  weight_path = open("/tmp/weight.txt", "r").read().strip()
  if weight_path == "":
    raise FileNotFoundError("Model has no weight file, please finish training first")
  weight_name = Path(weight_path).name
  # command does not fail if index is missing, this is intended
  !zip "/content/{model_name}.zip" "{model_name}/{weight_name}" "{model_name}/{model_name}.index"

if Path("/content/drive").is_mount():
  !mkdir -p "{EXPORT_PATH}"
  !mv "/content/{model_name}.zip" "{EXPORT_PATH}" && echo "Exported model to {EXPORT_PATH}/{model_name}.zip"
else:
  !echo "Drive not mounted, exporting model to /content/{model_name}.zip"